# Описание проекта

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Задачи:
 - подготовка данных.
 - категоризация данных.
 - ответы на вопросы: есть ли зависимость между семейным положением/количеством детей/доходом/целью кредита и вероятностью его возвращения в срок. 


## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd                      # импорт библиотеки pandas
from pymystem3 import Mystem             # импорт библиотеки pymystem3
import warnings                          # импорт службы warnings
warnings.simplefilter("ignore")          # игнорируем предупреждения

In [2]:
data = pd.read_csv('...')                # чтение файла 
data.info()                              # вывод общей информации о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Из общей информации видны пропуски в столбцах 'days_employed' и 'total_income'. Кроме того, общий трудовой стаж в днях и ежемесячный доход имеют тип данных float, для расчётов будет достаточно целочисленных величин.

In [3]:
display(data.head(20)) # вывод первых 20 строк для восприятия, с чем работаю

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Из вывода первых 20 строк видно, что общий трудовой стаж в днях, в основном, отрицательное число, а если и не отрицательное - то слишком большое (людям меньше лет чем их трудовой стаж). Гипотеза: отрицательные числа возникают из-за того, что время обращения в банк < времени начала работы. Большие числа стажа для пенсионеров объясняются тем, что в столбце для них указаны часы выхода на пенсию, а не дни стажа. Начало отсчёта для пенсионеров ведётся от 01.01.1970. Гипотеза требует проверки - нужно связаться с предоставившим данные.

In [4]:
print(data['children'].value_counts()) # вывод уникальных значений столбца 'дети' и их количества

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


**Вывод**

Столбец с графой 'дети' содержит отрицательное число -1 и выпадающее значение 20. Если -1 явно сигнализирует об ошибке в данных, то информация по людям с 20 детьми требует уточнения у предоставившего данные сотрудника.

In [5]:
print(data.groupby(['income_type','gender'])['total_income'].describe()) # вывод сводной таблицы медианной зарплаты для
                                                    # разных типов занятости и пола

                         count           mean            std            min  \
income_type     gender                                                        
безработный     F          1.0  202722.511368            NaN  202722.511368   
                M          1.0   59956.991984            NaN   59956.991984   
в декрете       F          1.0   53829.130729            NaN   53829.130729   
госслужащий     F        962.0  155680.311468   82383.054881   29200.077193   
                M        350.0  212726.065675  119057.595718   39154.156961   
компаньон       F       2870.0  184221.811725  105880.606379   28702.812889   
                M       1706.0  233027.090448  159168.843841   42777.827594   
                XNA        1.0  203905.157261            NaN  203905.157261   
пенсионер       F       2806.0  134038.546168   77751.691242   20667.263793   
                M        637.0  150734.228922   89224.154404   21205.280566   
предприниматель F          1.0  499163.144947       

**Вывод**

Что касается медианных зарплат, то они оказались зависящими от типа занятости и пола, это должно быть учтено при заполнении пропусков.

In [6]:
data['income_type'].value_counts() # вывод уникальных значений столбца тип занятости и их количества

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

**Вывод**

Данные также указывают, что в банк обращаются, в основном, работающие люди и люди на пенсии. Доля предпринимателей, безработных и студентов минимальна. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
mean_days_employed = data[data['days_employed'] <= 0].groupby('dob_years')['days_employed'].mean() # определение 
                                                # среднего стажа в зависимости от возраста клиента
data['days_employed'] = data['days_employed'].fillna(1) # заполнение пропуска в стаже для того, чтобы длина столбца 
                                                    # стажа совпадала с длинами других столбцов
try:                                                # используем конструкцию try-exept
    for i in range(len(data['days_employed'])):     # идём по столбцу занятости
        if data['days_employed'].loc[i] > 0:        # если натыкаемся на нашу вставленную 1 или неадекватно большое число
            data['days_employed'].loc[i] = mean_days_employed.loc[data['dob_years'][i]] # замена на соотв. возрасту средний стаж
    data['days_employed'] = data['days_employed'] * (-1) # перезаписываем стаж как положительное число

    data['total_income'] = data['total_income'].fillna(-1) # заполнение пропуска в зарплате для того, чтобы длина 
                                                           # столбца c зарплатой совпадала с длинами других столбцов
    for i in range(len(data['total_income'])):             # идём по столбцу зарплаты
        if data['total_income'].loc[i] < 0:                # если натыкаемся на нашу вставленную -1
            data['total_income'].loc[i] = data.groupby(['income_type','gender'])['total_income'].median().loc[data['income_type'][i],data['gender'][i]]
                                                     # вставляем медианную зарплату в соответствии с типом занятости и полом
except:
    print('Пропуски заполнены, переходим к следующему шагу')
data.info()   # проверка, остались ли пропуски
data.head(30)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3280.806279,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Для заполнения пропусков в стаже, а также замены неадекватных значении учитывалась зависимость среднего стажа от возраста. Пропуски в зарплате заполнялись с учётом медианных зарплат для разных типов занятости и разных полов.

Поскольку пропуски в стаже появлялись вместе с пропусками в средней зарплате, а это значения числовые, которые предоставляются по справке 2-НДФЛ(возможно), то выдвигаю гипотезу: клиенты просто не притащили справку в банк, но в базу их занесли и сказали: "Справку потом донесёте") 

### Замена типа данных

In [8]:
data['days_employed'] = (data['days_employed'] / 365).astype('int')  # переводим стаж в года и в тип данных int
data['total_income'] = (data['total_income'] / 1000).astype('int')   # измеряем зарплату в тысячах руб., 
                                                                    #меняем тип данных на int
data.rename(columns={'days_employed': 'years_employed'}, inplace=True) # переименовываем столбец со стажем
data.info()     # проверка типов данных 
data.head()   # вывод первых 5 строчек датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
years_employed      21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование
4,0,8,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу


**Вывод**

Для замены типа данных был использован метод astype(), потому что он позволяет менять тип данных на указанный в скобках, в отличие от метода to_numeric(), который приводит данные к типу float.

Цифры в столбцах со стажем и зарплатой округлились до целого остатка. Для решения поставленных задач округление по правилам математики не требуется.

### Обработка дубликатов

In [9]:
print(data['education'].value_counts()) # вывод уникальных значений в столбце образование
data['education'] = data['education'].str.lower()                # перевод всего столбца 'education' к нижн. регистру
print(f'Количество явных дубликатов до обработки: {data.duplicated().sum()}') # подсчёт дубликатов до обработки
data = data.drop_duplicates().reset_index(drop=True) # удаление дубликатов
print(f'Количество дубликатов после обработки: {data.duplicated().sum()}') # проверка, остались ли одинаковые строки

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
Количество явных дубликатов до обработки: 76
Количество дубликатов после обработки: 0


**Вывод**

Обработка дубликатов началась с определения неявных повторений в "образовании". После приведения этого столбца к нижнему регистру были удалены явные дубликаты. 

Гипотезы причины появления дубликатов: 
1. Кто-то уже работал с датафреймом и в процессе работы перезаписал некоторые строки, в том числе строки, в которых одинаковое полученное образование, но использован разный регистр.
2. Возможно, что человек пришёл в филиал крупного банка и оставил заявку. Заявка полетела в общую базу всех филиалов. По каким-то причинам система решила, что данные пришли из двух филиалов и записала дубликат.
3. Оператор, вводя данные клиента сначала их сохранила ('Save'), а потом сделала 'Save as'.

### Лемматизация

In [10]:
m = Mystem()       # переобозначение анализатора русского языка
lemmas_raw = []    # создание пустой строки, куда мы будем записывать лемматизированные словосочетания
for i in data['purpose']:           # цикл по столбцу цель кредита
    lemmas = m.lemmatize(i)         # лемматизация текущей строки
    lemmas_raw.append(lemmas)       # добавляем в наш список список лемм
data['lem_purpose'] = lemmas_raw    # добавляем столбик со списком лемм

data.head(20)    # проверка результата

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья,"[покупка, , жилье, \n]"
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья,"[покупка, , жилье, \n]"
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,8,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255,покупка жилья,"[покупка, , жилье, \n]"
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135,образование,"[образование, \n]"
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


**Вывод**

Лемматизация была проведена поэлементно для каждого обращения. После чего был сформирован столбец, содержащий список лемм каждого обращения.

### Категоризация данных

In [11]:
print(data['purpose'].unique())   # смотрим уникальные значения целей кредита 
def purpose_group(purpose_lem):   # функция, которая с помощью получившихся списков лемм относит цель 
                                  # к определённой категории
    
    if 'жилье' in purpose_lem or 'недвижимость' in purpose_lem:
        if 'коммерческий' in purpose_lem:
            return 'коммерческая недвижимость'
        if 'ремонт' in purpose_lem:
            return 'ремонт'
        return 'жилье'       
    if 'автомобиль' in purpose_lem:
        return 'автомобиль'
    if 'свадьба' in purpose_lem:
        return 'свадьба'
    if 'образование' in purpose_lem:
        return 'образование'
    if 'коммерческий' in purpose_lem:
        return 'коммерческая недвижимость'
    
    return 'цель кредита не определена'

data['purpose_grouped'] = data['lem_purpose'].apply(purpose_group)           # применяем функцию к столбцу с леммами 
print('Список уникальных целей кредитования :' , data['purpose_grouped'].unique()) # проверка группировки

income_median = data['total_income'].median()          # считаем медианную зарплату
income_poor = 0.6 * income_median                      # определяем границу бедности как 60% медианы
income_rich = 2 * income_median                        # определяем границу бедности как 200% медианы
def income_group(income):                              # функция, которая определяет категорию достатка
    if income <= income_poor:
        return 'бедный'
    if income >= income_rich:
        return 'зажиточный'
    return 'средний класс'
data['income_grouped'] = data['total_income'].apply(income_group) # формируем новый столбик с категорией достатка
data.head(20)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
Список уникальных целей креди

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,purpose_grouped,income_grouped
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья,"[покупка, , жилье, \n]",жилье,средний класс
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,средний класс
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья,"[покупка, , жилье, \n]",жилье,средний класс
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование,"[дополнительный, , образование, \n]",образование,средний класс
4,0,8,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний класс
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255,покупка жилья,"[покупка, , жилье, \n]",жилье,средний класс
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240,операции с жильем,"[операция, , с, , жилье, \n]",жилье,средний класс
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135,образование,"[образование, \n]",образование,средний класс
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,средний класс
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье,средний класс


**Вывод**

Категоризация проводилась как по категориальным величинам, так и по количественным. Категоризация выполнялась по 5 возможным целям кредита: 'жилье' 'автомобиль' 'образование' 'свадьба' 'коммерческая недвижимость'
 'ремонт'. Эти цели требуют различных размеров кредита. Категоризация по уровню достатка состояла из выделения 3 категорий : 'бедные', 'средний класс', 'зажиточные'. Выделение количественных категорий основывалось на медианной зарплате. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
data_grouped_by_children = data.groupby('children')['debt'].count() # группируем людей по количеству детей
print(data_grouped_by_children)
probability_debt_by_children = data.groupby('children')['debt'].sum() / data_grouped_by_children # вычисление  
                                                                              # доли должников по группам
print(probability_debt_by_children.sort_values(ascending=False))        #Вывод долей в порядке убывания
                                                            # Поскольку измеренные доли отличаются на единицы %, то
                                                            # репрезентативной будет выборка с числом людей > 100
data_grouped_by_children = probability_debt_by_children.loc[data_grouped_by_children > 100] # выбор только 
                                                                                        # репрезентативных данных    
data_grouped_by_children.sort_values(ascending=False) # вывод данных для анализа, в порядке убывания доли невозврата

children
-1        47
 0     14087
 1      4808
 2      2051
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64
children
 20    0.105263
 4     0.097561
 2     0.094588
 1     0.092346
 3     0.081818
 0     0.075460
-1     0.021277
 5     0.000000
Name: debt, dtype: float64


children
2    0.094588
1    0.092346
3    0.081818
0    0.075460
Name: debt, dtype: float64

**Вывод**

Зависимость между наличием детей и возвратом кредита в срок есть, но она не монотонна. Минимальный процент невозврата у бездетных. С ростом числа детей растёт и процент невозврата, исключение - люди с 3 детьми. Возможная причина такого поведения - с появлением 3 ребёнка семья становится многодетной и включаются программы для многодетных, которые уменьшают вероятность невозврата кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
data_grouped_by_status = data.groupby('family_status')['debt'].count() # группировка данных по семейному положению
probability_debt_by_status = data.groupby('family_status')['debt'].sum() / data_grouped_by_status # вычисление доли 
                                                                    # должников по группам 
print(data_grouped_by_status) # вывод сргуппированных по семейному положеннию людей на экран
probability_debt_by_status.sort_values(ascending=False) # вывод данных для анализа, в порядке убывания доли невозврата


family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             958
гражданский брак          4150
женат / замужем          12336
Name: debt, dtype: int64


family_status
Не женат / не замужем    0.097509
гражданский брак         0.093494
женат / замужем          0.075470
в разводе                0.071130
вдовец / вдова           0.065762
Name: debt, dtype: float64

**Вывод**

В случае с семейным положением все группы семейного положения можно считать репрезентативными. Самыми ненадёжными оказались люди не женатые \ не замужем. А самыми надёжными люди, у которых вторая половина уже умерла. Видимо, на возврат влияет и возраст клиента, потому-что при сортировке по убыванию доли невозврата прослеживается 'жизненный путь' человека.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
data['debt_status'] = 'должник'                    # создаём столбец статуса человека
data.loc[data.debt == 0, 'debt_status'] = 'чист'   # и заполняем в соответствии со столбцом debt

data_pivot = data.pivot_table(index='income_grouped', columns='debt_status', values='debt', aggfunc='count') # создаём
                                # по разным категориям дохода граждан

data_pivot['non-return'] = data_pivot['должник']/ (data_pivot['должник']+data_pivot['чист']) # добавляем к сводной
                                                                        # таблице долю невозврата в каждой категории
data_pivot

debt_status,должник,чист,non-return
income_grouped,,,
бедный,230,2803,0.075833
зажиточный,120,1578,0.070671
средний класс,1391,15327,0.083204


**Вывод**

Согласно данным, зависимость между уровнем дохода и возвратом кредита в срок есть. Самым 'ненадёжным' оказался средний класс граждан. Самым низким невозвратом кредитов обладают зажиточные люди.

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
data_grouped_by_purpose = data.groupby('purpose_grouped')['debt'].count() # группировка данных по целям
probability_debt_by_purpose = data.groupby('purpose_grouped')['debt'].sum() / data_grouped_by_purpose # вычисление доли
                                                                # невозврата для каждой группы
print(data_grouped_by_purpose)
probability_debt_by_purpose.sort_values(ascending=False) # сортировка

purpose_grouped
автомобиль                   4305
жилье                        8890
коммерческая недвижимость    1311
образование                  4013
ремонт                        607
свадьба                      2323
Name: debt, dtype: int64


purpose_grouped
автомобиль                   0.093612
образование                  0.092200
свадьба                      0.080069
коммерческая недвижимость    0.075515
жилье                        0.072891
ремонт                       0.057661
Name: debt, dtype: float64

**Вывод**

Исследование указывает, что самой большой вероятностью невозврата обладают кредиты взятые на автомобиль и образование, а самыми 'надёжными' являются кредиты на операции с недвижимостью (ремонт,покупка). Ситуация объясняется тем, что образование и машина не являются благами первой необходимости, в отличие от жилья. Люди более серьёзно относятся к тем вещам, без которых их жизнь будет затруднена, а без машины и образования можно спокойно существовать.

## Шаг 4. Общий вывод

Исследование показало, что:
1. Количество детей влияет на вероятность возврата кредита:
    * Самыми надёжными заёмщиками являются бездетные
    * Самыми ненадёжными - люди с 2 детьми
    
2. Семейное положение влияет на вероятность возврата кредита:
    * Самыми надёжными заёмщиками являются люди, у которых вторая половина уже умерла
    * Самыми ненадёжными оказались не женатые/ не замужем
3. Уровень дохода влияет на вероятность возврата кредита:
    * Самыми надёжными заёмщиками являются  зажиточные люди
    * Самыми ненадёжными заёмщиками являются люди среднего класса и это неожиданно
4. Цели кредита влияют на его возврат в срок:
    * Самыми 'безопасными' целями для банка являются операции с недвижимостью (ремонт, покупка жилой недвижимости)
    * Самыми 'опасными' являются цели связанные с предметами не первой необходимости (автомобиль, образование) 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.